In [ ]:
#pip install webdriver-manager selenium 

In [58]:
group_names = ["ScrapingTesting" , "G1 Branch"]

In [57]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from selenium.webdriver.common.by import By
import csv
import time

chrome_options = Options()
chrome_options.add_argument("user-data-dir=selenium") 
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get("https://web.whatsapp.com/")

In [59]:
def split_date_time_name(a):
    pattern = r'\[(.*?), (.*?)\] (.*?):'
    match = re.match(pattern, a)
    return(match.group(1) ,  match.group(2) ,  match.group(3) )


def extract_messages_to_csv():
    x_path_of_all_text_messages = '//div[@role = "row"]//div[contains(@class , "copyable-text")]'
    message_elements = driver.find_elements(By.XPATH, x_path_of_all_text_messages )
    for message_element in message_elements : 
        text_data = message_element.get_attribute("data-pre-plain-text")
        text_message_xpath = './/span[@dir = "ltr"]/span'
        text_message = message_element.find_element(By.XPATH, text_message_xpath)
        splitted_data = split_date_time_name(text_data)
        append_data = [group_name , splitted_data[1] , splitted_data[2] , splitted_data[0] ,text_message.text  ]
        writer.writerow(append_data)
        

        
with open('whatsapp_messages.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Group Name" , "Date" , "Sender Name", "Time", "Message"])
    for group_name in group_names : 
        x_path = '//span[@dir = "auto" and @title =' +'"'+ group_name + '"]' 
        chathead_element = driver.find_element(By.XPATH, x_path )
        chathead_element.click()
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', chathead_element)
        extract_messages_to_csv()
    file.flush()

In [ ]:
driver.quit()